In [0]:
!pip install -q transformers

In [0]:
!/opt/bin/nvidia-smi
!cat /proc/cpuinfo

Fri Apr 24 14:39:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from __future__ import print_function
from transformers import pipeline
from google.colab import files
from transformers import BertConfig
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import json
import math
import random

In [0]:
target_medicine='news_content_translate_en'
society=[]
politics=[]
sports=[]
technology=[]
economy=[]

## Load Data from mutiple files

In [0]:

with open("economy.json","r") as f:
    temp=json.loads(f.read())
for x in temp:
    if 'news_content_translate_en' in x:
        if 'news_content_translate_cn' in x:
            economy.append(x)

with open("politics.json","r") as f:
    temp=json.loads(f.read())
for x in temp:
    if 'news_content_translate_en' in x:
        if 'news_content_translate_cn' in x:
            politics.append(x)

with open("society.json","r") as f:
    temp=json.loads(f.read())
for x in temp:
    if 'news_content_translate_en' in x:
        if 'news_content_translate_cn' in x:
            society.append(x)

with open("sports.json","r") as f:
    temp=json.loads(f.read())
for x in temp:
    if 'news_content_translate_en' in x:
        if 'news_content_translate_cn' in x:
            sports.append(x)

with open("technology.json","r") as f:
    temp=json.loads(f.read())
for x in temp:
    if 'news_content_translate_en' in x:
        if 'news_content_translate_cn' in x:
            technology.append(x)

FileNotFoundError: ignored

## Load data from on file

In [0]:
with open("less_society.json","r") as f:
    temp = json.loads(f.read())
    for x in temp:
        if 'news_content_translate_en' in x and 'news_content_translate_cn' in x:
            if x['news_subject'] == 'politics':
                politics.append(x)
            elif x['news_subject'] == 'economy':
                economy.append(x)
            elif x['news_subject'] == 'society':
                society.append(x)
            elif x['news_subject'] == 'sports':
                sports.append(x) 
            elif x['news_subject'] == 'technology':
                technology.append(x) 

In [0]:
politics[0]
economy[0]
society[0]
sports[0]
technology[0]

{'id': '5e8a149a02f74d1837931953',
 'news_about_china': False,
 'news_content_translate_cn': '来自澳大利亚默多克研究所的一组研究人员今天宣布，他们将在感染covid-19疫苗的医疗专业人员中进行测试，以验证在减轻疾病症状方面的有效性。\n墨尔本默多克研究所的研究人员o n e解释说：“虽然卡介苗最初是用来治疗肺结核的，但每年仍要给超过1.3亿婴儿使用，但卡介苗也会增加人体内基本免疫物质的消耗。”，法新社引用。\n这项临床试验将涉及澳大利亚医院约4000名卫生专业人员，以验证这种疫苗是否有能力减轻SARS-CoV-2疾病的症状。\n研究小组组长奈杰尔·柯蒂斯解释说，如果卡介苗按预期发挥作用，受感染的医疗专业人员中的covid-19将“症状的频率和严重程度有所降低”。\n这些大规模试验也将在荷兰、德国或英国等其他国家进行。\n导致dim-19大流行的新型冠状病毒已经感染了全球50.5万人，其中约23000人死亡。\n在感染病例中，至少108.900人被认为已治愈。\n去年12月在中国出现后，疫情在全球蔓延，导致世界卫生组织（WHO）宣布出现大流行局面。\n欧洲大陆有近275000人感染，16000人死亡，是目前出现病例最多的大陆，意大利是世界上死亡人数最多的国家，迄今为止，在80.539例病例中有8.165人死亡。\n西班牙是死亡人数最多的第二个国家，死亡人数为4.089人，确诊感染病例在56.188人之间。是 啊。\n除香港和澳门外，中国有81285例（超过7.4万人恢复）和3287例死亡记录。周三，他报告了6例死亡和67例新病例，全部来自国外，当时该国在瘫痪两个月后开始恢复正常。\n受影响最大的国家是伊朗，仅次于意大利、西班牙和中国，报告死亡2.234例（29.406例），法国死亡1.696例（29.155例），美国死亡1.178例（81.321例）。\n迄今为止，非洲大陆已有87例因新型冠状病毒死亡的记录，超过46个国家的3200例。\n一些国家采取了特殊措施，包括隔离和关闭边境。',
 'news_content_translate_en': 'A team of researchers from the Murdoch Institute in Australia a

## 7. Projection - Features Extraction 

In [0]:
def get_string(target_medicine,data):
    l=[]
    for x in data:
        l.append(x[target_medicine])
    return l

In [0]:
import numpy as np
def get_content_features(docl):
    config = BertConfig.from_pretrained('bert-base-cased', num_labels=5)
    get_features = pipeline('feature-extraction','bert-base-cased',config=config,device=0)
    output = []
    data_batch_size = 15
    for i in range(0, math.ceil(len(docl)/data_batch_size)):
        output.extend(
            get_features(*
                        (docl[i * data_batch_size:min(i*data_batch_size + data_batch_size, len(docl))])
                        , pad_to_max_length=True
            )
        )
    return output
#np.savez("./test.npz" ,np.array(output))   # (Samples, Tokens, Vector Size)
#files.download("./test.npz")

In [0]:
get_class_id = {
    'politics': 0,
    'society': 1,
    'technology': 2,
    'economy': 3,
    'sports': 4,
}


In [0]:
def get_batched_data_in_a_go():
    data_l=[]
    test_l=[]
    y_train=[]
    y_test=[]

    random.shuffle(economy)
    i=0
    for x in economy: 
        if i<=0.7*len(economy): 
            data_l.append(x)
            y_train.append(get_class_id[x['news_subject']])
        else: 
            test_l.append(x)
            y_test.append(get_class_id[x['news_subject']])
        i=i+1

    # random.shuffle(entertainment)
    # i=0
    # for x in entertainment: 
    #     if i<=0.7*len(entertainment): 
    #         data_l.append(x)
    #         y_train.append(get_class_id[x['news_subject']])
    #     else: 
    #         test_l.append(x)
    #         y_test.append(get_class_id[x['news_subject']])
    #     i=i+1
    
    # random.shuffle(humanity)
    # i=0
    # for x in humanity: 
    #     if i<=0.7*len(humanity): 
    #         data_l.append(x)
    #         y_train.append(get_class_id[x['news_subject']])
    #     else: 
    #         test_l.append(x)
    #         y_test.append(get_class_id[x['news_subject']])
    #     i=i+1

    random.shuffle(politics)
    i=0
    for x in politics: 
        if i<=0.7*len(politics): 
            data_l.append(x)
            y_train.append(get_class_id[x['news_subject']])
        else: 
            test_l.append(x)
            y_test.append(get_class_id[x['news_subject']])
        i=i+1

    random.shuffle(society)
    i=0
    for x in society: 
        if i<=0.7*len(society): 
            data_l.append(x)
            y_train.append(get_class_id[x['news_subject']])
        else: 
            test_l.append(x)
            y_test.append(get_class_id[x['news_subject']])
        i=i+1

    random.shuffle(sports)
    i=0
    for x in sports: 
        if i<=0.7*len(sports): 
            data_l.append(x)
            y_train.append(get_class_id[x['news_subject']])
        else: 
            test_l.append(x)
            y_test.append(get_class_id[x['news_subject']])
        i=i+1

    random.shuffle(technology)
    i=0
    for x in technology: 
        if i<=0.7*len(technology): 
            data_l.append(x)
            y_train.append(get_class_id[x['news_subject']])
        else: 
            test_l.append(x)
            y_test.append(get_class_id[x['news_subject']])
        i=i+1

    data_l=get_string(target_medicine,data_l)
    test_l=get_string(target_medicine,test_l)
    x_train=np.array(get_content_features(data_l))
    x_train=x_train.reshape(len(data_l),512,768,1)
    x_test = np.array(get_content_features(test_l))
    x_test = x_test.reshape(len(test_l),512,768,1)

    y_train=np.array(y_train).reshape(len(y_train),1)
    y_test = np.array(y_test).reshape(len(y_test),1)
    
    return x_train,y_train,x_test,y_test

In [0]:
def func_train(train_data,train_label,test_data,test_label):
    model = models.Sequential()
    model.add(layers.Dropout(0.5,input_shape=(512,768,1)))
    model.add(layers.Conv2D(32,(2,768),activation='relu'))
    model.add(layers.MaxPooling2D((48, 1)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(5,activation='relu'))
    model.summary()
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(train_data,train_label,batch_size=32,epochs=60,validation_data=(test_data,test_label))
    model.save('model.h5')
    return

In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU
import tensorflow_addons as tfa

def non_linear_train(train_data, train_label, test_data, test_label):
    tf.keras.backend.clear_session()

    def gelu(x):
        return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
    get_custom_objects().update({'gelu': Activation(gelu)})
    
    inputs=keras.Input(shape=(512,768,1))
    conv_2d_layer1=layers.Conv2D(64,(2,768),activation='relu')(inputs);
    conv_2d_layer2=layers.Conv2D(64,(3,768),activation='relu')(inputs);
    conv_2d_layer3=layers.Conv2D(64,(4,768),activation='relu')(inputs);
    conv_2d_layer4=layers.Conv2D(64,(5,768),activation='relu')(inputs);
    max_pool_a=layers.MaxPooling2D((511,1))(conv_2d_layer1)
    max_pool_b=layers.MaxPooling2D((510,1))(conv_2d_layer2)
    max_pool_c=layers.MaxPooling2D((509,1))(conv_2d_layer3)
    max_pool_d=layers.MaxPooling2D((508,1))(conv_2d_layer4)
    x=layers.concatenate([max_pool_a,max_pool_b,max_pool_c,max_pool_d])
    flatten=layers.Flatten()(x)
    drop_out=layers.Dropout(0.5)(flatten)
    #dense_1=layers.Dense(64,activation='relu')(drop_out)
    #dense_2=layers.Dense(25,activation='relu')(dense_1)
    outputs=layers.Dense(5,activation='re')(drop_out)
    model=keras.Model(inputs=inputs, outputs=outputs, name='non_linear')
    model.summary()
    model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit(train_data,train_label,epochs=60,validation_data=(test_data,test_label))


Using TensorFlow backend.


In [0]:
(train_data,train_label,test_data,test_label)=get_batched_data_in_a_go()

In [0]:
test_label.shape

(175, 1)

In [0]:
non_linear_train(train_data,train_label,test_data,test_label)

Model: "non_linear"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 768, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 511, 1, 64)   98368       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 510, 1, 64)   147520      input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 509, 1, 64)   196672      input_1[0][0]                    
_________________________________________________________________________________________

In [0]:
func_train(train_data,train_label,test_data,test_label)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_11 (Dropout)         (None, 512, 768, 1)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 511, 1, 32)        49184     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 10, 1, 32)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 320)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 320)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                20544     
_________________________________________________________________
dense_12 (Dense)             (None, 5)                

In [0]:
files.download('./model.h5')